<a href="https://colab.research.google.com/github/erees1/alignment-jam/blob/main/Whisper_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Whisper attention investigation

# Setup

In [83]:
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")

Running as a Jupyter notebook - intended for development only!


In [84]:
import torch
import torch.nn as nn

from collections import defaultdict

## Data

In [85]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import datasets
import torch

In [86]:
# load dummy dataset and read soundfiles
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

Found cached dataset librispeech_asr_dummy (/home/edwardr/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


In [87]:
m = model.model.decoder.layers[0].self_attn

# Infering the model code

In [88]:
# Have a look at the source code for the forward method
import inspect
def print_method(func):
    lines = inspect.getsource(func)
    print(lines)
print_method(m.forward)

    def forward(
        self,
        hidden_states: torch.Tensor,
        key_value_states: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        """Input shape: Batch x Time x Channel"""

        # if key_value_states are provided this layer is used as a cross-attention layer
        # for the decoder
        is_cross_attention = key_value_states is not None

        bsz, tgt_len, _ = hidden_states.size()

        # get query proj
        query_states = self.q_proj(hidden_states) * self.scaling
        # get key, value proj
        if is_cross_attention and past_key_value is not None:
            # reuse k,v, cross_attentions
            key_states = past_key_value[0]
            value_states = past_key_

In [89]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

In [90]:
print_method(model.generate)

    @torch.no_grad()
    def generate(
        self,
        inputs: Optional[torch.Tensor] = None,
        max_length: Optional[int] = None,
        min_length: Optional[int] = None,
        do_sample: Optional[bool] = None,
        early_stopping: Optional[bool] = None,
        num_beams: Optional[int] = None,
        temperature: Optional[float] = None,
        penalty_alpha: Optional[float] = None,
        top_k: Optional[int] = None,
        top_p: Optional[float] = None,
        typical_p: Optional[float] = None,
        repetition_penalty: Optional[float] = None,
        bad_words_ids: Optional[Iterable[int]] = None,
        force_words_ids: Optional[Union[Iterable[int], Iterable[Iterable[int]]]] = None,
        bos_token_id: Optional[int] = None,
        pad_token_id: Optional[int] = None,
        eos_token_id: Optional[int] = None,
        length_penalty: Optional[float] = None,
        no_repeat_ngram_size: Optional[int] = None,
        encoder_no_repeat_ngram_size: Optional[

# Attention Plots

In [91]:
def register_hooks(model):
    # Activation hook to get attention scores
    attention_scores = defaultdict(list)
    def getActivation(name):
        # the hook signature
        def hook(module, input, output):
            attention_scores[name].append(output[1].detach())
        return hook

    hooks = []
    for i, layer in enumerate(model.model.decoder.layers): 
        h = layer.self_attn.register_forward_hook(getActivation(f'layer-{i}.self_attn'))
        hooks.append(h)
        h = layer.encoder_attn.register_forward_hook(getActivation(f'layer-{i}.encoder_attn'))
        hooks.append(h)
    return hooks, attention_scores

In [92]:
def get_num_features(audio_len):
    n_mfccs = ((audio_len - 400) // processor.feature_extractor.hop_length + 1)
    attn_len = n_mfccs // 2 # Whisper has a conv layer with stride of 2 at the start of the encoder
    return  attn_len

In [93]:
def run_model_with_hooks(model, data):
    hooks, attention_scores = register_hooks(model)
    audio_len = len(data)
    feat_len = get_num_features(audio_len)

    input_features = processor(data, return_tensors="pt", sampling_rate=16000, return_attention_mask = True).input_features 
    # Generate logits
    predicted_ids = model.generate(input_features, decoder_input_ids = torch.tensor([[50258]]), output_attentions=True)
    # take argmax and decode
    transcription_as_list = [processor.decode(p) for p in predicted_ids.flatten()]
    [h.remove() for h in hooks]
    return transcription_as_list, attention_scores, feat_len

In [94]:
def process_outputs(attention_scores, feat_len):
    """
    Produce a dictionary where key is the layer name and each value is of shape
    N, N, D where N is the attention seq length and D is number of heads.
    """

    output = {}
    len_of_output_seq = len(list(attention_scores.values())[0])
    for k, v in attention_scores.items():
        # v is list of attention scores where v[i] is 
        # the attention for the ith token
        # v[i] has shape (1, n_heads, 1, attn_len)
        if 'encoder_attn' in k:
            # len of input_sequence + len of output seq required to plot
            # attn pairs
            seq_len = feat_len + len_of_output_seq  
        else:
            seq_len = len_of_output_seq
        
        n_heads = v[-1].shape[1]

        full_shape = torch.zeros((seq_len, seq_len, n_heads))
        for token_id, score in enumerate(v):
            expand = torch.zeros((seq_len, n_heads))
            s = score.reshape((n_heads, score.shape[-1]))
            s = s.permute(-1, 0)   # (s seq_len, num_heads)
            if 'encoder' in k:
                expand[:s.shape[0]] = s
                
                full_shape[token_id - len_of_output_seq] = expand
            else:
                expand[:s.shape[0]] = s  # (seq_len, num_heads)
                full_shape[token_id, ...] = expand
        output[k] = full_shape
    return output

In [95]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [96]:
transcript_as_list, raw_attention_scores, feat_len = run_model_with_hooks(model, ds[0]["audio"]["array"])

# reduce length of the attention scores - modifies in place!
window_size = 3
stride = 3
remove_padding = True
if remove_padding:
    input_tokens_to_plot = feat_len
else:
    input_tokens_to_plot = raw_attention_scores['layer-0.encoder_attn'][0][0, 0, 0].shape[-1]

from torch.nn import AvgPool1d
avg = AvgPool1d(window_size, stride, 0)
def sum_pool_1d(x):
    return avg(x) * avg.kernel_size[0]

avg = AvgPool1d(window_size, stride, 0)
for layer_name, attention in raw_attention_scores.items():
    if 'encoder' in layer_name:
        new_a = []
        for token_id, attn_at_token in enumerate(attention):
            a = sum_pool_1d(attn_at_token[:,:,:,:input_tokens_to_plot].squeeze()).unsqueeze(0).unsqueeze(-2)
            attention[token_id] = a
        raw_attention_scores[layer_name] = attention
new_input_tokens_len = a.shape[-1]
print(new_input_tokens_len)

/home/edwardr/.pyenv/bin/versions/3.10.3/envs/whisper/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


97


In [97]:
attention_scores = process_outputs(raw_attention_scores, new_input_tokens_len)

In [98]:
import pysvelte
n_layers=1
for k, v in list(attention_scores.items())[:n_layers*2]:
    print()
    print(f"{k}")
    print("-" * 20)
    if 'encoder_attn' in k:
        tokens=['*'] * new_input_tokens_len + transcript_as_list[:-1]
    else:
        tokens = transcript_as_list[:-1]
    pysvelte.AttentionMulti(tokens=tokens, attention=v).show()



layer-0.self_attn
--------------------



layer-0.encoder_attn
--------------------


In [99]:
from whisper.audio import load_audio
def get_attn_scores_for_file(file_path):

    transcript_as_list, raw_attention_scores, feat_len = run_model_with_hooks(model, load_audio(file_path))

    # reduce length of the attention scores - modifies in place!
    window_size = 2
    stride = 2
    remove_padding = True
    if remove_padding:
        input_tokens_to_plot = feat_len
    else:
        input_tokens_to_plot = raw_attention_scores['layer-0.encoder_attn'][0][0, 0, 0].shape[-1]

    from torch.nn import AvgPool1d
    avg = AvgPool1d(window_size, stride, 0)
    def sum_pool_1d(x):
        return avg(x) * avg.kernel_size[0]

    avg = AvgPool1d(window_size, stride, 0)
    for layer_name, attention in raw_attention_scores.items():
        if 'encoder' in layer_name:
            new_a = []
            for token_id, attn_at_token in enumerate(attention):
                a = sum_pool_1d(attn_at_token[:,:,:,:input_tokens_to_plot].squeeze()).unsqueeze(0).unsqueeze(-2)
                attention[token_id] = a
            raw_attention_scores[layer_name] = attention
    new_input_tokens_len = a.shape[-1]
    print(new_input_tokens_len)
    attention_scores = process_outputs(raw_attention_scores, new_input_tokens_len)

    import pysvelte
    n_layers=1
    for k, v in list(attention_scores.items())[:n_layers*2]:
        print()
        print(f"{k}")
        print("-" * 20)
        if 'encoder_attn' in k:
            tokens=['* '] * new_input_tokens_len + transcript_as_list[:-1]
        else:
            tokens = transcript_as_list[:-1]
        pysvelte.AttentionMulti(tokens=tokens, attention=v).show()


In [100]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
# run_model_with_hooks(model, load_audio('/data/testsets/en/raw/en-US-3play-Education/audio/2285524.wav'))

In [119]:
!wget https://drive.google.com/u/0/uc?id=14R1MGTozskNpVXWhkt6ZkmSbc_9DKuLU&export=download -O audio.wav

zsh:1: no matches found: https://drive.google.com/u/0/uc?id=14R1MGTozskNpVXWhkt6ZkmSbc_9DKuLU
zsh:1: command not found: -O


In [116]:
!ls -lh

total 3.7M
-rw-r--r-- 1 edwardr cantab  72K Nov 13 11:59 14R1MGTozskNpVXWhkt6ZkmSbc_9DKuLU
-rw-r--r-- 1 edwardr cantab  72K Nov 13 12:00 14R1MGTozskNpVXWhkt6ZkmSbc_9DKuLU.1
-rw-r--r-- 1 edwardr cantab  72K Nov 13 12:00 14R1MGTozskNpVXWhkt6ZkmSbc_9DKuLU.2
-rw-r--r-- 1 edwardr cantab   16 Nov 12 21:14 README.md
-rw-r--r-- 1 edwardr cantab 4.0M Nov 13 11:54 Whisper_Attention.ipynb
-rw-r--r-- 1 edwardr cantab 1.3K Nov 13 12:00 audio.wav


In [118]:
get_attn_scores_for_file('audio.wav')

RuntimeError: Failed to load audio: ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-

In [101]:
get_attn_scores_for_file('/data/testsets/en/whisper_testsets/short_form/callhome/audio/en_4622-A.wav')

/home/edwardr/.pyenv/bin/versions/3.10.3/envs/whisper/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


750

layer-0.self_attn
--------------------



layer-0.encoder_attn
--------------------


In [ ]:
# get_attn_scores_for_file('/data/testsets/en/raw/en-US-3play-Education/audio/2199375.wav')

In [102]:
!ffprobe /data/testsets/en/whisper_testsets/short_form/callhome/audio/en_4622-A.wav

ffprobe version 3.4.11-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-l

In [103]:
!ffprobe /data/testsets/en/whisper_testsets/short_form/callhome/audio/sw_4507-B.wav

ffprobe version 3.4.11-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-l

In [105]:
!ffprobe '/data/testsets/en/whisper_testsets/short_form/callhome/audio/en_6183-A.wav'

ffprobe version 3.4.11-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-l

In [106]:
!ffprobe /data/testsets/en/whisper_testsets/short_form/callhome/audio/en_4966-A.wav

ffprobe version 3.4.11-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-l

In [ ]:
# get_attn_scores_for_file('/data/testsets/en/raw/en-US-3play-Online_Video/audio/2303220.wav')

## Whisper API

In [1]:
import whisper

model = whisper.load_model("medium")

In [10]:
import torch
import numpy as np
import random
import os
def set_seeds(seed=42, fully_deterministic=True):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.deterministic = False

    if fully_deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
        os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"
        torch.use_deterministic_algorithms(True)

set_seeds(fully_deterministic=True)

In [14]:
set_seeds()
# result = model.transcribe('/data/testsets/en/whisper_testsets/short_form/callhome/audio/en_4622-A.wav')

In [37]:
!ffprobe '/data/testsets/en/whisper_testsets/short_form/callhome/audio/en_4622-A.wav'

ffprobe version 3.4.11-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-l

In [ ]:
from huggingface

In [22]:
result
tokens = []
for segment in result["segments"]:
    tokens.extend(segment['tokens'])
transcript_as_list = [processor.decode(p) for p in predicted_ids.flatten()]

In [24]:
from whisper.tokenizer import get_tokenizer

In [31]:
tokenizer = get_tokenizer(True, language="en", task='transcription')

In [36]:
tokenizer.decode(tokens[0])

''

In [ ]:
def run_model_with_hooks(model, filepath):
    hooks, attention_scores = register_hooks(model)
    audio_len = len(data)
    feat_len = get_num_features(audio_len)

    model.transcribe('/data/testsets/en/raw/en-US-3play-Education/audio/2199375.wav')['text']
    # Generate logits
    predicted_ids = model.generate(input_features, decoder_input_ids = torch.tensor([[50258]]), output_attentions=True)
    # take argmax and decode
    transcription_as_list = [processor.decode(p) for p in predicted_ids.flatten()]
    [h.remove() for h in hooks]
    return transcription_as_list, attention_scores, feat_len